In [ ]:
import torch
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.io as pio
pio.renderers.default='notebook_connected'
sns.set()
import random
import numpy as np
from torch.nn.functional import conv1d
import plotly.express as px
from tqdm import tqdm
import time

In [ ]:
def generate_jump(p, n, s, Norm):
    ### Generate Delta, the direction of the change point located at tau1 ###

    Delta = 2*torch.bernoulli(torch.tensor(p*[1/2])).reshape(-1, 1) - 1

    ### Sparsify Delta
    sparse_loc = torch.bernoulli(torch.tensor(p*[1- s/p])).bool()
    Delta[sparse_loc] = 0
    return (Norm/Delta.norm()) * Delta

def generate_wcs(p, n, s, Norm, r=None):
    '''generates worst case shape of time series, that is with two change-points.'''
    ### Choose the scale r randomly ###
    r = int(torch.randint(1, n//2, size=(1,))) if r is None else r

    ### Choose the location of the first change point ###
    tau1 = torch.randint(0, n - r, size=(1,))
    tau2 = tau1 + r

    Delta = generate_jump(p, n, s, Norm)
    Theta = torch.zeros((p,n))
    Theta[:, tau1:tau2] = Delta
    return Theta

def generate_K(p, n, s, Norm, K, samples=1):
    Theta = torch.zeros(size=(samples,p,n)) #samples is for faster monte carlo
    change_points = torch.randperm(n)[:K]
    for tau in change_points:
        for sample in range(samples):
            Theta_single_cp = torch.zeros_like(Theta)
            Theta_single_cp[sample, :, tau:] = generate_jump(p, n, s, Norm)
            Theta.add_(Theta_single_cp)
    return Theta


def ending(r):
    return (None if r==1 else -r+1)

def logs_0(p, n, r, delta):
    return np.maximum(int(np.log2(np.log2(n/(r*delta)))), 1)
def logs_m(p, n, r, delta):
    gamma_r = np.log2(n/(r*delta))
    logsm = np.maximum(int(np.log2(np.sqrt(p*gamma_r)/(np.maximum(1, np.log2(p) - gamma_r)))), 1)
    return np.minimum(logsm, int(np.log2(p)))

def generate_grid(p, n, delta= 0.05):
#CompleteGrid = list(range(n//2))
    SemiDyadicGrid = [2**i for i in range(int(np.log2(n)))]
    Grid = {}
    for r in SemiDyadicGrid:
        Grid[r] = [2**i - 1 for i in range(logs_m(p, n, r, delta) + 1)]
        if 2**logs_m(p, n, r, delta) != p:
            Grid[r].append(p - 1)
    return Grid
def compute_cusums(Ys, r):
    samples, p, n = Ys.shape
    Weights = torch.zeros((1, 1, r)) + 1
    Convolutions = conv1d(Ys.reshape(samples*p, 1, n), Weights).reshape(samples, p, -1)
    ConvolutionsFilled = torch.zeros_like(Ys)
    end = ending(r)
    ConvolutionsFilled[:, :, :end] = Convolutions
    Convolutions = ConvolutionsFilled
    #Cusums = CusumsFilled
    Cusums = torch.zeros_like(Convolutions)
    Cusums[:, :, r:end] = ((Convolutions[:, :, r:end] - Convolutions[:, :, :-2*r + 1])
                            /np.sqrt(2*r))
    return Cusums
def compute_statistics_r(Y, r, Grid):
    Cusums = compute_cusums(Y, r)
    CusumsSquared = (Cusums**2).sort(dim=1, descending=True)[0] #sort along dimension R^p
    PartialNorms = CusumsSquared.cumsum(dim=1)
    Stats = PartialNorms[:, Grid[r], :]
    return Stats
def compute_statistics(Y, Grid, showbar=True):
    Stats = {}
    for r in Grid:
        if showbar:
            print(f'computing stats for r = {r}         ', end = '\r')
        Stats[r] = compute_statistics_r(Y, r, Grid)
    return Stats
    
def compute_thresholds_r(Grid, r, Stats_r, delta=0.05):
    Expects_r = Stats_r[:, :, r:ending(r)].mean(dim=(0,2))
    delta_rs = delta/(len(Grid)*len(Grid[r]))
    Thresholds_r = Stats_r[:,:,r:ending(r)].max(dim=2)[0].quantile(1-delta_rs, dim=0)
    return Thresholds_r, Expects_r
def compute_thresholds(Grid, Stats, delta=0.05, showbar=True):
    Thresholds = {}
    Expects = {}
    for r in Grid:
        if showbar:
            print(f'computing thresholds for r = {r}         ', end = '\r')
        Thresholds[r], Expects[r] = compute_thresholds_r(Grid, r, Stats[r], delta=delta)
    return Thresholds, Expects

In [ ]:
np.log2(1000)

In [ ]:
torch.manual_seed(1)
p, n, s, samples = 100, 100, 50, 100
Signals = generate_K(p, n, s, Norm=1, K=10, samples=samples)
Noise = torch.randn((samples, p, n))
Ys = Signals + Noise
Y = Ys[0]
Signal = Signals[0]
i = 4
px.scatter({'data' : Y[i, :], 'signal' : Signal[i, :]} )

In [ ]:
Cusums = compute_cusums(Signals, 5)
A = compute_cusums(Noise, 5)[:,:,5:ending(5)]
print('moyenne des cusums sur du bruit : ', (A**2).mean())
px.line({'signal' : Signals[0, 12, :], 'cusums' : Cusums[0, 12, :]})


In [ ]:
p, n, samples = 10000, 10, 100
Noise = torch.randn((samples, p, n))
Grid = generate_grid(p, n)

In [ ]:
Stats = compute_statistics(Noise, Grid)

In [ ]:
compute_thresholds(Grid, Stats)

In [ ]:
def estimate_constants_lists(
    pmax, nmax, sigma=1, samples=30, delta=0.1, quantile=0.7, pmin=1, nmin=1
):
    C_dict = {}
    Cdense_list = []
    Csparse_list = []
    Cverysparse_list = []
    for p in [2**i for i in range(int(np.log2(pmin)), int(np.log2(pmax)) + 1)]:
        C_dict[p] = {}
        for n in [2**i for i in range(int(np.log2(nmin)), int(np.log2(nmax)) + 1)]:
            C_dict[p][n] = {}
            Noise = torch.randn((samples, p, n))
            Grid = generate_grid(p, n)
            for r in Grid:
                print(f"p = {p} ; n = {n} ;  r = {r}         ", end="\r")
                Stats_r = compute_statistics_r(Noise, r, Grid)
                Thresholds_r, Expects_r = compute_thresholds_r(Grid, r, Stats_r)
                UB_verysparse = np.log2(n / (r * delta))
                s = torch.Tensor(Grid[r])
                UB_sparse = s * np.log2(p / s)
                UB_dense = np.sqrt(p * np.log2(n / (r * delta)))
                C_dict[p][n][r] = Thresholds_r - Expects_r
                print(len(C_dict[p][n][r]))
                Cverysparse_list.extend(
                    C_dict[p][n][r][: logs_0(p, n, r, delta)] / UB_verysparse
                )
                Csparse_list.extend(
                    (C_dict[p][n][r] / UB_sparse)[logs_0(p, n, r, delta): -1]
                )
                Cdense_list.append(C_dict[p][n][r][-1] / UB_dense)
    return tuple(map(torch.Tensor, (Cverysparse_list, Csparse_list, Cdense_list)))


def estimate_constants(constants_lists):
    def f(C_list):
        return C_list.sort(descending=True)[0][
            len(C_list) // 5
        ].mean()  # mean of the 20% highest const

    return tuple(map(f, constants_lists))


constants_lists = estimate_constants_lists(
    pmax=1000, nmax=1000, pmin=999, nmin=999, samples=100, delta=0.3
)


In [ ]:
constants_lists

In [ ]:
estimate_constants(constants_lists)

In [ ]:

estimate_constants(constants_lists)

In [ ]:
estimate_constants(constants_lists)

In [ ]:
Cvsp

In [ ]:
torch.Tensor(Cdense)

In [ ]:
a = torch.arange(27, dtype=float).reshape(3,3,3)
a.max(dim=0)

In [ ]:
A = torch.randn((10, 3))
print(torch.quantile(A, 0.5, dim=0, interpolation='higher'))
print(A)

In [ ]:
def CallibrateThresholds(p, n, Grid, samples=100, delta=0.99):
    delta_r = delta/len(Grid) #This callibration is valid only for semi complete grid
    thresholds = {}
    for r in Grid:
        Noise = torch.randn((samples, p, n))
        Cusums = compute_cusums(Noise, r)
        torch.quantile(a, 0.6, interpolation='higher')